# SCXA to H5AD conversion

Experiments fo convert single cell expression atlas files (https://www.ebi.ac.uk/gxa/sc/experiments/E-CURD-134/downloads) to AnnData format.

In [13]:
## IMPORT depedencies 
import os
import scanpy as sc
import anndata as ad

import urllib.request as request
import zipfile

Download and prepare SCXA data. Download operation may take some time (file size is approx. 137.5 MB).

You can manually download and copy E-CURD-134-normalised-files.zip to project root folder to speed-up the process.

In [16]:
if not os.path.exists(os.path.join(os.getcwd(), "E-CURD-134-normalised-files.zip")):
    # Download
    request.urlretrieve("https://www.ebi.ac.uk/gxa/sc/experiment/E-CURD-134/download/zip?fileType=normalised&accessKey=", "E-CURD-134-normalised-files.zip")

if not os.path.exists(os.path.join(os.getcwd(), "downloads/matrix.mtx")):
    # Unzip
    with zipfile.ZipFile("E-CURD-134-normalised-files.zip", 'r') as zip_ref:
        zip_ref.extractall(os.path.join(os.getcwd(), "downloads/"))
    # rename files
    os.rename(os.path.join(os.getcwd(), "downloads/E-CURD-134.aggregated_filtered_normalised_counts.mtx"), os.path.join(os.getcwd(), "downloads/matrix.mtx"))
    os.rename(os.path.join(os.getcwd(), "downloads/E-CURD-134.aggregated_filtered_normalised_counts.mtx_cols"), os.path.join(os.getcwd(), "downloads/barcodes.tsv"))
    os.rename(os.path.join(os.getcwd(), "downloads/E-CURD-134.aggregated_filtered_normalised_counts.mtx_rows"), os.path.join(os.getcwd(), "downloads/genes.tsv"))

print('Experiment data is ready.')

Experiment data is ready.


In [ ]:
ann_data = sc.read_10x_mtx('downloads/', var_names='gene_ids')
ann_data.obs.head(4)

""
SAMN13703199-AAACCTGAGCAGGTCA
SAMN13703199-AAACCTGAGCTTCGCG
SAMN13703199-AAACCTGAGGCTATCT
SAMN13703199-AAACCTGAGGTGCTAG


In [ ]:
ad.AnnData.write_h5ad(ann_data, "out/E-CURD-134.h5ad", compression="gzip")